In [173]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
import scipy.optimize as opt
from selenium.webdriver.common.by import By
import numpy as np
import astropy.units as u

In [121]:
path = '/Users/wendy/_Lib/chromedriver_mac_arm64/chromedriver'

In [122]:
driver = webdriver.Chrome(path)


/var/folders/1c/5vhp7f916wv452z62sq58lzc0000gn/T/ipykernel_39231/2459958106.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [123]:
driver.get('https://smm.misis.ru/CPW-resonator-coupling/')


In [124]:
def set_epsilon(epsilon):
    epsi = driver.find_element(By.ID, 'epsilon')
    epsi.clear()
    epsi.send_keys('11.45')

In [125]:
def single_cpw_impedance(pin, gap):
    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    AB = input_box.find_element(By.NAME, 'AB')
    BC = input_box.find_element(By.NAME, 'BC')
    CD = input_box.find_element(By.NAME, 'CD')

    AB.clear()
    AB.send_keys(gap)

    CD.clear()
    CD.send_keys(gap)

    BC.clear()
    BC.send_keys(pin)


    impedance_box = driver.find_element(By.ID, 'impedance-data')
    impedance = impedance_box.get_attribute('innerHTML')



    string = impedance
    match = re.search(r'Z0:(\d+\.\d+)', string)

    if match:
        number = float(match.group(1))
        # print(number)
    else:
        print("No match found.")


    return number

    

In [126]:
set_epsilon(11.45)

In [127]:
single_cpw_impedance(15.95, 9)

50.00560320940848

In [128]:
def find_optimal_gap(pin):
    def f(gap):
        time.sleep(0.01)
        return single_cpw_impedance(pin, gap)-50
    gap_val = opt.brentq(f, 0, 10)
    return round(gap_val, 2)

In [129]:
#for the transmission line, the optimal gap is:
find_optimal_gap(15.95)

9.0

In [130]:
#for the resonator:
find_optimal_gap(12.4)

6.99

In [131]:
resonator_pin = 12.4
transmission_line_pin = 15.95
resonator_gap = find_optimal_gap(resonator_pin)
transmission_line_gap = find_optimal_gap(transmission_line_pin)

After getting optimal values for the individual transmissionline/resonators, we optimize the coupling between them.

In [132]:
def set_up_structure(coupling_space,resonator_pin = resonator_pin,resonator_gap=resonator_gap,transmission_line_pin=transmission_line_pin, transmission_line_gap=transmission_line_gap):
    button = driver.find_element(By.ID, 'add-conductor')
    button.click()
    button.click()
    time.sleep(0.5)
    conductor1 = driver.find_element(By.ID, 'tc3')
    conductor1.click()
    conductor1.click()
    time.sleep(0.5)

    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    AB = input_box.find_element(By.NAME, 'AB')
    BC = input_box.find_element(By.NAME, 'BC')
    CD = input_box.find_element(By.NAME, 'CD')
    DE = input_box.find_element(By.NAME, 'DE')
    EF = input_box.find_element(By.NAME, 'EF')
    FG = input_box.find_element(By.NAME, 'FG')
    GH = input_box.find_element(By.NAME, 'GH')

    AB.clear()
    AB.send_keys(transmission_line_gap)

    CD.clear()
    CD.send_keys(transmission_line_gap)

    BC.clear()
    BC.send_keys(transmission_line_pin)
    time.sleep(0.5)

    
    DE.clear()
    DE.send_keys(coupling_space)

    EF.clear()
    EF.send_keys(resonator_gap)

    FG.clear()
    FG.send_keys(resonator_pin)

    GH.clear()
    GH.send_keys(resonator_gap)
    

In [170]:
def change_gap(coupling_space):
    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    DE = input_box.find_element(By.NAME, 'DE')
    DE.clear()
    DE.send_keys(coupling_space)

In [133]:
def set_up_resonator():
    ztl1 = driver.find_element(By.ID, 'Zt1_short')
    ztl1.click()
    ztl2 = driver.find_element(By.ID, 'Zt2_open')
    ztl2.click()

    SC_coupler_len = driver.find_element(By.ID, 'lc')
    SC_coupler_len.clear()
    SC_coupler_len.send_keys('0.12')

    SC_coupler_l1 = driver.find_element(By.ID, 'l1')
    SC_coupler_l1.clear()
    SC_coupler_l1.send_keys('0')

    SC_coupler_l2 = driver.find_element(By.ID, 'l2')
    SC_coupler_l2.clear()
    SC_coupler_l2.send_keys('4')

In [134]:
set_up_structure(5)



In [148]:
set_up_resonator()

In [136]:
def find_resonator_length(freq):
    def f(length):
        time.sleep(0.01)
        SC_coupler_len = driver.find_element(By.ID, 'lc')
        SC_coupler_len.clear()
        SC_coupler_len.send_keys(length)
        return single_cpw_impedance(12.4, 9)-freq
    length_val = opt.brentq(f, 0, 10, xtol=0.001)
    return round(length_val, 2)

In [137]:
def find_resonator_specs():

    table = driver.find_element(By.ID, 'resonances')
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')

    # Extracting column names from HTML table headers
    cols = []
    for th in soup.find_all('th'):
        cols.append(th.text.strip())
        
    # Extracting table rows from HTML table body
    data = []
    for tr in soup.find_all('tr')[1:]:
        row = []
        for td in tr.find_all('td'):
            row.append(td.text.strip())
        data.append(row)

    # Creating pandas dataframe from table rows and columns
    df = pd.DataFrame(data, columns=cols)

    return df
    
    

In [142]:
freq = float(find_resonator_specs()['Frequency, GHz'][0])

In [149]:
def find_freq(len):
    SC_coupler_l2 = driver.find_element(By.ID, 'l2')
    SC_coupler_l2.clear()
    SC_coupler_l2.send_keys(len)

    return float(find_resonator_specs()['Frequency, GHz'][0])

In [190]:
def find_resonator_length(freq):
    def f(length):
        time.sleep(0.01)
        SC_coupler_len = driver.find_element(By.ID, 'l2')
        SC_coupler_len.clear()
        SC_coupler_len.send_keys(length)
        return float(find_resonator_specs()['Frequency, GHz'][0])-freq
    length_val = opt.brentq(f, 0, 10, xtol=0.001)
    return round(length_val, 2)

    

In [151]:
l = find_resonator_length(6.4)

In [158]:
freq_len = {}
for freqs in np.arange(6.4,8,0.2):
    freq_len[freqs] = find_resonator_length(freqs)

In [159]:
freq_len

{6.4: 4.57,
 6.6000000000000005: 4.43,
 6.800000000000001: 4.3,
 7.000000000000001: 4.17,
 7.200000000000001: 4.05,
 7.400000000000001: 3.94,
 7.600000000000001: 3.83,
 7.800000000000002: 3.73}

In [181]:
def find_Q(gap,freq):

    change_gap(gap)
    
    length = find_resonator_length(freq)
    SC_coupler_len = driver.find_element(By.ID, 'l2')
    SC_coupler_len.clear()
    SC_coupler_len.send_keys(length)

    
    return float(find_resonator_specs()['Quality factor'][0])

    


In [182]:
Q = find_Q(1,6.4)

In [183]:
kappa = 2*np.pi*freq*u.GHz/Q

In [184]:
kappa.to(u.MHz)

<Quantity 1.26695002 MHz>

In [187]:
def find_optimal_gap(freq):
    def kappa(gap):
        time.sleep(0.01)
        return (2*np.pi*freq*u.GHz/find_Q(gap,freq)-1*u.MHz).to(u.MHz).value
    gap_val = opt.brentq(kappa, 0, 10, xtol=0.001)
    return round(gap_val, 2)

In [188]:
find_optimal_gap(6.4)

1.35

In [192]:
gaps = {}

for freq in np.arange(6.4,8,0.2):
    gaps[freq] = find_optimal_gap(freq)

In [193]:
gaps

{6.4: 1.35,
 6.6000000000000005: 1.68,
 6.800000000000001: 2.03,
 7.000000000000001: 2.41,
 7.200000000000001: 2.8,
 7.400000000000001: 3.19,
 7.600000000000001: 3.6,
 7.800000000000002: 4.0}

In [197]:
dat = pd.DataFrame(gaps, index = [0])
dat.to_csv('gaps_um.csv')

In [166]:
conductor1 = driver.find_element(By.ID, 'tc3')

In [60]:
conductor1.click()

In [61]:
conductor1.click()

In [62]:
input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
AB = input_box.find_element(By.NAME, 'AB')
BC = input_box.find_element(By.NAME, 'BC')
CD = input_box.find_element(By.NAME, 'CD')
DE = input_box.find_element(By.NAME, 'DE')
EF = input_box.find_element(By.NAME, 'EF')
FG = input_box.find_element(By.NAME, 'FG')
GH = input_box.find_element(By.NAME, 'GH')

In [63]:
AB.clear()
AB.send_keys('9')
CD.clear()
CD.send_keys('9')

BC.clear()
BC.send_keys('15.95')

DE.clear()
DE.send_keys('5')

EF.clear()
EF.send_keys('7.4')

FG.clear()
FG.send_keys('12.5')

GH.clear()
GH.send_keys('7.4')



In [64]:
epsi = driver.find_element(By.ID, 'epsilon')
epsi.clear()
epsi.send_keys('11.45')

In [67]:
SC_coupler_len = driver.find_element(By.ID, 'lc')
SC_coupler_len.clear()
SC_coupler_len.send_keys('0.12')

SC_coupler_l1 = driver.find_element(By.ID, 'l1')
SC_coupler_l1.clear()
SC_coupler_l1.send_keys('0')

SC_coupler_l2 = driver.find_element(By.ID, 'l2')
SC_coupler_l2.clear()
SC_coupler_l2.send_keys('4')



In [117]:
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# Extracting column names from HTML table headers
cols = []
for th in soup.find_all('th'):
    cols.append(th.text.strip())
    
# Extracting table rows from HTML table body
data = []
for tr in soup.find_all('tr')[1:]:
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    data.append(row)

# Creating pandas dataframe from table rows and columns
df = pd.DataFrame(data, columns=cols)

print(df)

   n Frequency, GHz Linewidth, MHz Decay time, μs Quality factor
0  1        7.28689           0.11          2.899          66362
1  2       21.86082           0.98          0.324          22249
2  3       36.43517           2.70          0.118          13504
3  4       51.01020           5.20          0.061           9814
4          65.58615           8.40          0.038           7813
